In [27]:
##파이썬##

In [15]:
import os
import openai
import sys
import json
import tiktoken
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.chat_message_histories import SQLChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory

# 환경 변수 설정
os.environ["OPENAI_API_KEY"] = 'sk-'
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# 토큰화 설정
tokenizer = tiktoken.encoding_for_model("gpt-3.5-turbo")

def tiktoken_len(text):
    tokens = tokenizer.encode(text)
    return len(tokens)

# PDF 로드 및 텍스트 분할
try:
    loader = PyPDFLoader("D:/[24]ICT_Practice/practice_file/(2024)포트미스_가이드북.pdf")
    pages = loader.load_and_split()
    print("PDF 로드 및 분할 성공")
except Exception as e:
    print(f"PDF 로드 및 분할 실패: {e}")

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50, length_function=tiktoken_len)
texts = text_splitter.split_documents(pages)

# 임베딩 설정
try:
    model_name = "jhgan/ko-sbert-nli"
    model_kwargs = {'device': 'cpu'}
    encode_kwargs = {'normalize_embeddings': True}
    hf = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs)
    docsearch = Chroma.from_documents(texts, hf)
    print("임베딩 설정 성공")
except Exception as e:
    print(f"임베딩 설정 실패: {e}")

# OpenAI 모델 설정
openai_model = ChatOpenAI(model_name="gpt-3.5-turbo", streaming=False, temperature=0)

# SQLChatMessageHistory 객체 생성
def get_chat_message_history(session_id):
    try:
        history = SQLChatMessageHistory(session_id=session_id, connection_string="sqlite:///sqlite.db")
        print(f"SQLChatMessageHistory 생성 성공: {session_id}")
        return history
    except Exception as e:
        print(f"SQLChatMessageHistory 생성 실패: {e}")
        raise e

# 채팅 프롬프트 템플릿 설정
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

# 체인 설정
chain = prompt | openai_model
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_chat_message_history,
    input_messages_key="question",
    history_messages_key="history"
)

# 질의응답 체인 설정
try:
    qa = RetrievalQA.from_chain_type(
        llm=openai_model,
        chain_type="stuff",
        retriever=docsearch.as_retriever(search_type="mmr", search_kwargs={'k': 3, 'fetch_k': 10}),
        return_source_documents=True
    )
    print("질의응답 체인 설정 성공")
except Exception as e:
    print(f"질의응답 체인 설정 실패: {e}")

# 사용자 입력 처리 함수
def get_response(input_text, session_id):
    try:
        config = {"configurable": {"session_id": session_id}}
        chat_response = chain_with_history.invoke({"question": input_text}, config=config)
        print(f"응답 생성 성공: {chat_response}")
        return chat_response
    except Exception as e:
        print(f"응답 생성 실패: {e}")
        raise e

# 메인 함수
if __name__ == "__main__":
    input_text = sys.argv[1]
    session_id = sys.argv[2]  # 세션 ID를 명령줄 인수로 받음
    try:
        chat_response = get_response(input_text, session_id)
        
        # chat_response 구조 출력
        print(f"chat_response 구조: {chat_response}")
        
        # chat_response의 'result'를 가져오는 부분 수정
        if isinstance(chat_response, dict) and "content" in chat_response:
            chat_message = chat_response["content"]
        else:
            chat_message = str(chat_response)

        output = {
            "status": 200,
            "message": "채팅 응답 성공",
            "body": {
                "chatMessage": chat_message
            }
        }
    except Exception as e:
        output = {
            "status": 404,
            "message": "채팅 응답 실패",
            "body": {
                "error": str(e)
            }
        }
    
    print(json.dumps(output, ensure_ascii=False, indent=4))


PDF 로드 및 분할 성공
임베딩 설정 성공
질의응답 체인 설정 성공
SQLChatMessageHistory 생성 성공: C:\Users\유승미\AppData\Roaming\jupyter\runtime\kernel-cdbe03a7-aae3-42a6-9df3-43c41d3276e8.json
응답 생성 성공: content='It seems like there may be a misunderstanding. Please provide more information or let me know how I can assist you.' response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 95, 'total_tokens': 118}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-d27716ac-7628-4e70-bc45-0e9f269fa783-0' usage_metadata={'input_tokens': 95, 'output_tokens': 23, 'total_tokens': 118}
chat_response 구조: content='It seems like there may be a misunderstanding. Please provide more information or let me know how I can assist you.' response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 95, 'total_tokens': 118}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-d277

In [18]:
import os
import openai
import sys
import json
import tiktoken
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.chat_message_histories import SQLChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory

# 환경 변수 설정
os.environ["OPENAI_API_KEY"] = 'sk-'
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# 토큰화 설정
tokenizer = tiktoken.encoding_for_model("gpt-3.5-turbo")

def tiktoken_len(text):
    tokens = tokenizer.encode(text)
    return len(tokens)

# PDF 로드 및 텍스트 분할
loader = PyPDFLoader("D:/[24]ICT_Practice/practice_file/(2024)포트미스_가이드북.pdf")
pages = loader.load_and_split()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50, length_function=tiktoken_len)
texts = text_splitter.split_documents(pages)

# 임베딩 설정
model_name = "jhgan/ko-sbert-nli"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
hf = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs)
docsearch = Chroma.from_documents(texts, hf)

# OpenAI 모델 설정
openai_model = ChatOpenAI(model_name="gpt-3.5-turbo", streaming=False, temperature=0)

# SQLChatMessageHistory 객체 생성
def get_chat_message_history(session_id):
    return SQLChatMessageHistory(session_id=session_id, connection_string="sqlite:///sqlite.db")

# 채팅 프롬프트 템플릿 설정
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

# 체인 설정
chain = prompt | openai_model
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_chat_message_history,
    input_messages_key="question",
    history_messages_key="history"
)

# 질의응답 체인 설정
qa = RetrievalQA.from_chain_type(
    llm=openai_model,
    chain_type="stuff",
    retriever=docsearch.as_retriever(search_type="mmr", search_kwargs={'k': 3, 'fetch_k': 10}),
    return_source_documents=True
)

# 사용자 입력 처리 함수
def get_response(input_text, session_id):
    config = {"configurable": {"session_id": session_id}}
    try:
        chat_response = chain_with_history.invoke({"question": input_text}, config=config)
        if isinstance(chat_response, dict) and "content" in chat_response:
            chat_message = chat_response["content"]
        else:
            chat_message = str(chat_response)
        
        return {
            "status": 200,
            "message": "채팅 응답 성공",
            "body": {
                "chatMessage": chat_message
            }
        }
    except Exception as e:
        return {
            "status": 404,
            "message": "채팅 응답 실패",
            "body": {
                "error": str(e)
            }
        }

# Jupyter Notebook에서 직접 호출하기
def test_chat(input_text, session_id):
    result = get_response(input_text, session_id)
    return json.dumps(result, ensure_ascii=False, indent=4)


In [19]:
input_text = "외항선 입항신고에서 선박이 입항 전에 어 떤 정보를 신고해야 하는가?"
session_id = "session_1"
result = test_chat(input_text, session_id)
print(result)

{
    "status": 200,
    "message": "채팅 응답 성공",
    "body": {
        "chatMessage": "content='외항선 입항신고에서 선박이 입항 전에 다음과 같은 정보를 신고해야 합니다:\\n\\n1. 선박의 이름 및 국적\\n2. 선박의 IMO 번호 (선박 식별번호)\\n3. 선박의 총 톤수 및 길이\\n4. 선박의 운항 목적지 및 출발지\\n5. 선박의 ETA (예상 도착 시간)\\n6. 선박의 선원 수 및 승객 수\\n7. 선박이 운송하는 화물의 종류 및 양\\n8. 선박의 보안 상태 및 방역 상태\\n\\n이러한 정보는 해당 국가의 관련 규정에 따라 상세 내용이 달라질 수 있으니, 해당 규정을 확인하시는 것이 좋습니다.' response_metadata={'token_usage': {'completion_tokens': 252, 'prompt_tokens': 55, 'total_tokens': 307}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-a1286f16-d098-4416-b3a2-9dc2b3468bcc-0' usage_metadata={'input_tokens': 55, 'output_tokens': 252, 'total_tokens': 307}"
    }
}


In [22]:
input_text = "통합화물신고 절차는 어떻게 되나요?"
session_id = "session_1"
result = test_chat(input_text, session_id)
print(result)

{
    "status": 200,
    "message": "채팅 응답 성공",
    "body": {
        "chatMessage": "content='통합화물신고는 국제무역에서 수출 및 수입 화물에 대한 통관 절차를 간소화하고 효율화하기 위한 제도입니다. 일반적으로 통합화물신고 절차는 다음과 같습니다:\\n\\n1. 수출자 또는 수입자가 통관을 위해 통합화물신고를 작성합니다.\\n2. 통합화물신고에는 화물의 세부 정보, 운송 수단, 운송 업체, 수출자/수입자 정보 등이 포함됩니다.\\n3. 통합화물신고는 전자적으로 관세청 또는 관련 기관에 제출됩니다.\\n4. 관세청 또는 관련 기관은 제출된 통합화물신고를 검토하고 필요한 검사 및 검수를 실시합니다.\\n5. 검사 및 검수가 완료되면 통관이 승인되고 화물이 통관됩니다.\\n6. 통관이 완료되면 수출자 또는 수입자는 해당 화물을 수령하거나 발송할 수 있습니다.\\n\\n통합화물신고는 국가별로 절차와 요구사항이 다를 수 있으므로, 해당 국가의 관세청 또는 관련 기관의 지침을 따라야 합니다.' response_metadata={'token_usage': {'completion_tokens': 419, 'prompt_tokens': 340, 'total_tokens': 759}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-de5d6d8f-e296-4a3b-af63-946304b25082-0' usage_metadata={'input_tokens': 340, 'output_tokens': 419, 'total_tokens': 759}"
    }
}


In [26]:
import os
import openai
import sys
import json
import tiktoken
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.chat_message_histories import SQLChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory

# 환경 변수 설정
os.environ["OPENAI_API_KEY"] = 'sk-proj-1r91qhM6nM2lAvStku8vT3BlbkFJKzoqJnRd41ZRBojOQVLl'
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# 토큰화 설정
tokenizer = tiktoken.encoding_for_model("gpt-3.5-turbo")

def tiktoken_len(text):
    tokens = tokenizer.encode(text)
    return len(tokens)

# PDF 로드 및 텍스트 분할
loader = PyPDFLoader("D:/[24]ICT_Practice/practice_file/(2024)포트미스_가이드북.pdf")
pages = loader.load_and_split()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50, length_function=tiktoken_len)
texts = text_splitter.split_documents(pages)

# 임베딩 설정
model_name = "jhgan/ko-sbert-nli"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
hf = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs)
docsearch = Chroma.from_documents(texts, hf)

# OpenAI 모델 설정
openai_model = ChatOpenAI(model_name="gpt-3.5-turbo", streaming=False, temperature=0)

# SQLChatMessageHistory 객체 생성
def get_chat_message_history(session_id):
    return SQLChatMessageHistory(session_id=session_id, connection_string="sqlite:///sqlite.db")

# 채팅 프롬프트 템플릿 설정
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

# 체인 설정
chain = prompt | openai_model
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_chat_message_history,
    input_messages_key="question",
    history_messages_key="history"
)

# 질의응답 체인 설정
qa = RetrievalQA.from_chain_type(
    llm=openai_model,
    chain_type="stuff",
    retriever=docsearch.as_retriever(search_type="mmr", search_kwargs={'k': 3, 'fetch_k': 10}),
    return_source_documents=True
)

# 사용자 입력 처리 함수
def get_response(input_text, session_id):
    config = {"configurable": {"session_id": session_id}}
    try:
        chat_response = chain_with_history.invoke({"question": input_text}, config=config)
        if isinstance(chat_response, dict) and "content" in chat_response:
            chat_message = chat_response["content"]
        else:
            chat_message = str(chat_response)
        
        return {
            "status": 200,
            "message": "채팅 응답 성공",
            "body": {
                "chatMessage": chat_message
            }
        }
    except Exception as e:
        return {
            "status": 404,
            "message": "채팅 응답 실패",
            "body": {
                "error": str(e)
            }
        }

In [28]:
# 채팅 기록 조회 함수
def get_chat_history(session_id):
    history = get_chat_message_history(session_id)
    messages = history.load_messages()
    return messages

# 특정 질문 찾기
def find_question(session_id, question_text):
    messages = get_chat_history(session_id)
    for message in messages:
        if message.role == "human" and question_text in message.content:
            return message.content
    return None

# Jupyter Notebook에서 직접 호출하기
def test_chat(input_text, session_id):
    result = get_response(input_text, session_id)
    return json.dumps(result, ensure_ascii=False, indent=4)

In [29]:
# 예제 실행
input_text = "외항선 입항신고에서 선박이 입항 전에 어 떤 정보를 신고해야 하는가?"
session_id = "session_1"
result = test_chat(input_text, session_id)
print(result)

{
    "status": 200,
    "message": "채팅 응답 성공",
    "body": {
        "chatMessage": "content='외항선 입항신고에서 선박이 입항 전에 다음과 같은 정보를 신고해야 합니다:\\n\\n1. 선박의 이름 및 국적\\n2. 선박의 IMO 번호 (선박 식별번호)\\n3. 선박의 총 톤수 및 길이\\n4. 선박의 운항 목적지 및 출발지\\n5. 선박의 ETA (예상 도착 시간)\\n6. 선박의 선원 수 및 승객 수\\n7. 선박이 운송하는 화물의 종류 및 양\\n8. 선박의 보안 상태 및 방역 상태\\n\\n이러한 정보는 해당 국가의 관련 규정에 따라 상세 내용이 달라질 수 있으니, 해당 규정을 확인하시는 것이 좋습니다.' response_metadata={'token_usage': {'completion_tokens': 252, 'prompt_tokens': 805, 'total_tokens': 1057}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-4a56d8d7-5de0-4ac2-913e-fb83475f6f18-0' usage_metadata={'input_tokens': 805, 'output_tokens': 252, 'total_tokens': 1057}"
    }
}


In [30]:
# 질문 내용 찾기 예제 실행
found_question = find_question(session_id, "외항선 입항신고에서 선박이 입항 전에")
print(found_question)

AttributeError: 'SQLChatMessageHistory' object has no attribute 'load_messages'